In [ ]:
import time

import pandas as pd

import tokenizers
from predict import *
from retriever.indexing.es_client import ES
from utils.post_processing import load_classify_model
from utils.utils import *

In [ ]:
MODEL_PATH = "nguyenvulebinh/vi-mrc-large"
SCRIPTED_MODEL_PATH = "data/traced_mrc.pt"
CLASSIFY_MODEL_PATH = "data/model_classify"
BATCH_SIZE = 16

model_1 = load_scripted_qna_model(SCRIPTED_MODEL_PATH,
                                MODEL_PATH,
                                BATCH_SIZE,
                                device=0,
                                max_seq_len=128,
                                doc_stride=10,
                                handle_impossible_answer=False)
model_2 = load_scripted_qna_model(SCRIPTED_MODEL_PATH,
                                MODEL_PATH,
                                BATCH_SIZE,
                                device=0,
                                max_seq_len=384,
                                doc_stride=50,
                                handle_impossible_answer=False)
question_classifier = load_classify_model(CLASSIFY_MODEL_PATH)

In [ ]:
ES_HOST = "localhost"
ES_PORT = "9200"
ES_TITLE_INDEX = "wikipedia"
ES_DOCUMENT_INDEX = "paragraph"
TEST_PATH = "./data/e2eqa-train+public_test-v1/zac2022_testa_only_question.json"
MAP_ANSWER = "scripts/utils/map.json"

es_context = ES(ES_HOST, ES_PORT, ES_DOCUMENT_INDEX)
es_map_title = ES(ES_HOST, ES_PORT, ES_TITLE_INDEX)
data = read_file(TEST_PATH)
data = data['data']
mapper = read_file(MAP_ANSWER)

tok_class = tokenizer.get_class('corenlp')
tokenizer = tok_class()

In [ ]:
K = 3

results = []
all_predicted_time = []
for i, d in enumerate(data):
    tmp = {}
    _id = d['id']
    start_time = time.time()
    question = d['question']
    question = clean_question(question)
    contexts = get_contexts(question,es_context,tokenizer, k=K)
    contexts = " ".join(contexts)
    # answers = get_answers(model, question, contexts)
    answers_1, score_1 = get_answers(model_1, question, contexts)
    answers_2, score_2 = get_answers(model_2, question, contexts)
    answers = answers_1
    if score_2 > score_1:
        answers = answers_2
    tmp['answer'] = answers
    if answers:
        tmp['answer'] =  correct_answers(es_map_title, question, answers, question_classifier, mapper)
    end_time = time.time()
    predicted_time = int((end_time - start_time) * 1000)
    tmp['id'] = _id
    tmp['question'] = question
    results.append(tmp)
    all_predicted_time.append((_id, predicted_time))

write_file("jupyter_submission.json", {"data": results})
all_predicted_time_df = pd.DataFrame(all_predicted_time, columns=["fname", "time (millisecond)"])
all_predicted_time_df.to_csv("time_submission.csv")
tokenizer.shutdown()